In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer
from transformers import TFBertModel

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score


In [2]:
!pip install nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 7.3 MB/s eta 0:00:00


In [3]:
import nlpaug.augmenter.word.context_word_embs as aug
import nlpaug.augmenter.word as naw

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# BERT Model Training using negative review rating as imputed values

## Reading in the data

In [17]:
reviews = pd.read_csv("/content/drive/MyDrive/ISE540/Project/labelled_with_neg(rr).csv")
#reviews = pd.read_csv("/content/drive/MyDrive/ISE540/Project/labelled_with_zeros.csv")
reviews.head()

,Unnamed: 0,Review,Review_Rating,Service,Food Quality,Ambiance/Atmosphere,Wait Time,Price,Menu Variety
0,0,The New Mexican a style cuisine here is damn g...,5.0,4.0,5.0,-5.0,-5.0,-5.0,4.0
1,1,The Eastern European food is great but do not ...,5.0,-5.0,5.0,-5.0,5.0,-5.0,5.0
2,2,Great service and great scene. Two levels with...,5.0,5.0,-5.0,5.0,5.0,-5.0,-5.0
3,3,Had dinner here with my co-workers and this pl...,5.0,5.0,5.0,5.0,-5.0,-5.0,5.0
4,4,We have dined at Scoogi's many times and have ...,4.0,3.0,5.0,-4.0,-4.0,-4.0,5.0


In [18]:
reviews = reviews.drop(columns='Unnamed: 0')

In [19]:
reviews.head()

,Review,Review_Rating,Service,Food Quality,Ambiance/Atmosphere,Wait Time,Price,Menu Variety
0,The New Mexican a style cuisine here is damn g...,5.0,4.0,5.0,-5.0,-5.0,-5.0,4.0
1,The Eastern European food is great but do not ...,5.0,-5.0,5.0,-5.0,5.0,-5.0,5.0
2,Great service and great scene. Two levels with...,5.0,5.0,-5.0,5.0,5.0,-5.0,-5.0
3,Had dinner here with my co-workers and this pl...,5.0,5.0,5.0,5.0,-5.0,-5.0,5.0
4,We have dined at Scoogi's many times and have ...,4.0,3.0,5.0,-4.0,-4.0,-4.0,5.0


## Creating the train and test data in the start

In [20]:
X = reviews['Review']
y = reviews[["Service",	"Food Quality",	"Ambiance/Atmosphere",	"Wait Time",	"Price",	"Menu Variety"]]

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

In [22]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((4000,), (4000, 6), (1000,), (1000, 6))

In [23]:
X_input_ids = np.zeros((len(X_train), 256))
X_attn_masks = np.zeros((len(X_train), 256))

X_input_ids.shape, X_attn_masks.shape

((4000, 256), (4000, 256))

In [24]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [25]:
def generate_training_data(data, ids, masks, tokenizer):
  for i, text in tqdm(enumerate(data)):
    token = tokenizer.encode_plus(
      text,
      max_length=256,
      truncation=True,
      padding='max_length',
      add_special_tokens=True,
      return_tensors='tf'
    )
    ids[i, :] = token.input_ids
    masks[i, :] = token.attention_mask

  return ids, masks

In [26]:
## Training tokens
X_input_ids, X_attn_masks = generate_training_data(X_train, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [27]:
## Training labels
service_labels = np.abs(y_train['Service'].to_numpy())-1
service_labels = tf.keras.utils.to_categorical(service_labels, num_classes=5 )

food_qlt_labels = np.abs(y_train['Food Quality'].to_numpy())-1
food_qlt_labels = tf.keras.utils.to_categorical(food_qlt_labels, num_classes=5)

atmos_labels = np.abs(y_train['Ambiance/Atmosphere'].to_numpy())-1
atmos_labels = tf.keras.utils.to_categorical(atmos_labels, num_classes=5 )

wait_labels = np.abs(y_train['Wait Time'].to_numpy())-1
wait_labels = tf.keras.utils.to_categorical(wait_labels, num_classes=5 )

price_labels = np.abs(y_train['Price'].to_numpy())-1
price_labels = tf.keras.utils.to_categorical(price_labels, num_classes=5 )

menu_labels = np.abs(y_train['Menu Variety'].to_numpy())-1
menu_labels = tf.keras.utils.to_categorical(menu_labels, num_classes=5 )

## Train and Validation datasets

In [28]:
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, service_labels, food_qlt_labels, atmos_labels, wait_labels, price_labels, menu_labels))

In [29]:
def map_function(input_ids, attn_masks, service_labels, food_qlt_labels, atmos_labels, wait_labels, price_labels, menu_labels):
  return {
      'input_ids': input_ids,
      'attention_mask': attn_masks
  }, {
      'service_output': service_labels,
      'food_qlt_output': food_qlt_labels,
      'atmos_output': atmos_labels,
      'wait_output': wait_labels,
      'price_output': price_labels,
      'menu_output': menu_labels
  }

In [30]:
dataset = dataset.map(map_function)
dataset = dataset.shuffle(10000, seed=42, reshuffle_each_iteration=False).batch(16, drop_remainder=True)

In [31]:
p = 0.8
train_size = int((len(reviews)//16)*p)

train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

train_cardinality = tf.data.experimental.cardinality(train_dataset).numpy()
val_cardinality = tf.data.experimental.cardinality(val_dataset).numpy()

print(f"Number of samples in the train dataset: {train_cardinality * 16}")
print(f"Number of samples in the validation dataset: {val_cardinality * 16}")

Number of samples in the train dataset: 3984
Number of samples in the validation dataset: 16


## Test dataset

In [32]:
X_input_ids = np.zeros((len(X_test), 256))
X_attn_masks = np.zeros((len(X_test), 256))

X_input_ids.shape, X_attn_masks.shape

((1000, 256), (1000, 256))

In [33]:
X_input_ids, X_attn_masks = generate_training_data(X_test, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [34]:
test_dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks))

In [35]:
def map_function(input_ids, attn_masks):
  return {
      'input_ids': input_ids,
      'attention_mask': attn_masks
  }

In [36]:
test_dataset = test_dataset.map(map_function)
test_dataset = test_dataset.batch(100, drop_remainder=True)

## BERT model definition

In [37]:
bert_model = TFBertModel.from_pretrained('bert-base-cased')

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [42]:
# model creation
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attention_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embeds = bert_model.bert(input_ids, attention_mask=attention_masks)[1]
intermediate_layer_1 = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer_1')(bert_embeds)
#intermediate_layer_2 = tf.keras.layers.Dense(256, activation='relu', name='intermediate_layer_2')(intermediate_layer_1)
service_output = tf.keras.layers.Dense(5, activation='softmax', name='service_output')(intermediate_layer_1)
food_qlt_output = tf.keras.layers.Dense(5, activation='softmax', name='food_qlt_output')(intermediate_layer_1)
atmos_output = tf.keras.layers.Dense(5, activation='softmax', name='atmos_output')(intermediate_layer_1)
wait_output = tf.keras.layers.Dense(5, activation='softmax', name='wait_output')(intermediate_layer_1)
price_output = tf.keras.layers.Dense(5, activation='softmax', name='price_output')(intermediate_layer_1)
menu_output = tf.keras.layers.Dense(5, activation='softmax', name='menu_output')(intermediate_layer_1)

model = tf.keras.Model(inputs=[input_ids, attention_masks], outputs=[service_output, food_qlt_output, atmos_output, wait_output, price_output, menu_output])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 256)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 256)]                0         []                            
 )                                                                                                
                                                                                                  
 bert (TFBertMainLayer)      TFBaseModelOutputWithPooli   1083102   ['input_ids[0][0]',           
                             ngAndCrossAttentions(last_   72         'attention_mask[0][0]']      
                             hidden_state=(None, 256, 7                                     

## BERT model training

In [43]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')
loss_func = tf.keras.losses.CategoricalCrossentropy()

In [44]:
model.compile(optimizer=optim,
              loss={'service_output': loss_func, 'food_qlt_output': loss_func, 'atmos_output': loss_func, 'wait_output': loss_func, 'price_output': loss_func, 'menu_output': loss_func},
              metrics=['accuracy'])

In [45]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=3)

Epoch 1/3
249/249 [==============================] - 243s 822ms/step - loss: 6.9428 - service_output_loss: 1.1315 - food_qlt_output_loss: 1.1242 - atmos_output_loss: 1.1667 - wait_output_loss: 1.2058 - price_output_loss: 1.1834 - menu_output_loss: 1.1313 - service_output_accuracy: 0.5341 - food_qlt_output_accuracy: 0.5567 - atmos_output_accuracy: 0.4970 - wait_output_accuracy: 0.4955 - price_output_accuracy: 0.5103 - menu_output_accuracy: 0.5246 - val_loss: 4.9648 - val_service_output_loss: 0.7259 - val_food_qlt_output_loss: 0.6236 - val_atmos_output_loss: 0.9369 - val_wait_output_loss: 0.8600 - val_price_output_loss: 0.9841 - val_menu_output_loss: 0.8343 - val_service_output_accuracy: 0.6875 - val_food_qlt_output_accuracy: 0.8125 - val_atmos_output_accuracy: 0.5625 - val_wait_output_accuracy: 0.6250 - val_price_output_accuracy: 0.5000 - val_menu_output_accuracy: 0.6250
Epoch 2/3
249/249 [==============================] - 204s 819ms/step - loss: 5.2153 - service_output_loss: 0.8605 - f

## Model Saving

In [46]:
# model.save("Path has to be specified")

## Model Evaluation

In [ ]:
# load the saved model if necessary

# model = tf.keras.models.load_model("path has to be specified")

In [58]:
## Helper functions for caclculating accuracy

def acc(ground_truth, predictions):
  non_zero_positions = (ground_truth >= 0) & (predictions >= 0)
  correct_predictions = (ground_truth[non_zero_positions] == predictions[non_zero_positions])
  accuracy = correct_predictions.sum() / non_zero_positions.sum() if non_zero_positions.sum() > 0 else 0
  return accuracy


def multiclass_accuracy(y_true, y_pred):
  labels = ['Service', 'Food Quality', 'Atmosphere/Ambiance', 'Wait Time', 'Price', 'Menu Variety']

  sum_acc = 0

  for i in range(len(labels)):
    temp_acc = acc(y_true[:,i], y_pred[:,i])
    sum_acc += temp_acc
    print(f"{labels[i]} Accuracy: {temp_acc*100:.3f}%")

  mean_acc = sum_acc / 6

  print(f"Average accuracy: {mean_acc*100:.3f}%")


In [48]:
y_preds = model.predict(test_dataset)

10/10 [==============================] - 17s 2s/step


In [49]:
y_preds = np.array(y_preds)
y_preds.shape

(6, 1000, 5)

In [50]:
y_preds = np.argmax(y_preds, axis=2)
y_preds = y_preds.T
y_preds.shape

(1000, 6)

In [51]:
y_preds

array([[2, 2, 2, 2, 2, 2],
       [4, 4, 4, 4, 4, 4],
       [4, 4, 4, 4, 4, 4],
       ...,
       [3, 3, 3, 3, 3, 3],
       [4, 4, 4, 4, 4, 4],
       [4, 4, 4, 4, 4, 4]])

In [52]:
y_test.shape

(1000, 6)

In [53]:
y_test = y_test.to_numpy()

In [55]:
y_test = y_test - 1

In [56]:
y_test

array([[ 1.,  2.,  2., -4., -4.,  4.],
       [ 4.,  4., -6.,  3., -6.,  4.],
       [ 4.,  4.,  4., -6., -6., -6.],
       ...,
       [ 4.,  3.,  4., -5.,  3.,  4.],
       [ 4.,  3., -4., -4., -4.,  2.],
       [ 3.,  4.,  2., -6., -6., -6.]])

In [59]:
multiclass_accuracy(y_test, y_preds)

Service Accuracy: 62.069%
Food Quality Accuracy: 64.798%
Atmosphere/Ambiance Accuracy: 56.061%
Wait Time Accuracy: 47.887%
Price Accuracy: 54.307%
Menu Variety Accuracy: 66.061%
Average accuracy: 58.530%


# BERT Model Training using data augmention techniques

## Data Augmentation

In [47]:
reviews = pd.read_csv("/content/drive/MyDrive/ISE540/Project/labelled_with_zeros.csv")
reviews = reviews.drop(columns='Unnamed: 0')
reviews.head()

,Review,Review_Rating,Service,Food Quality,Ambiance/Atmosphere,Wait Time,Price,Menu Variety
0,The New Mexican a style cuisine here is damn g...,5.0,4.0,5.0,0.0,0.0,0.0,4.0
1,The Eastern European food is great but do not ...,5.0,0.0,5.0,0.0,5.0,0.0,5.0
2,Great service and great scene. Two levels with...,5.0,5.0,0.0,5.0,5.0,0.0,0.0
3,Had dinner here with my co-workers and this pl...,5.0,5.0,5.0,5.0,0.0,0.0,5.0
4,We have dined at Scoogi's many times and have ...,4.0,3.0,5.0,0.0,0.0,0.0,5.0


In [48]:
train, valid = train_test_split(reviews, test_size=0.20, random_state=42)

In [49]:
# focusing only on Food Quality category
food_train = train[['Review', 'Food Quality']]
food_train.head()

,Review,Food Quality
4227,My wife and I share a birthday and we decided ...,5.0
4676,Probably one of my favorite Brewhouses in Indy...,4.0
800,I stopped by a couple of weeks ago and their a...,4.0
3671,I just stopped in to have a beer while waiting...,0.0
4193,Plain and simple SMASHBURGER has good burgers....,5.0


In [50]:
food_train['Food Quality'].value_counts()

5.0    1801
4.0     653
3.0     462
0.0     432
2.0     338
1.0     314
Name: Food Quality, dtype: int64

In [9]:
augmenter = aug.ContextualWordEmbsAug(model_path='bert-base-uncased', action='insert')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [10]:
sample_text = food_train['Review'][0]
sample_text

"The New Mexican a style cuisine here is damn good. I love the breakfast, especially with a bloody mary. Yum! The green and red sauces are delicious but SPICY so if you don't like spice consider the sour cream green chili sauce, or half and half. The staff are pretty great too."

In [11]:
augmented_text = augmenter.augment(sample_text)
augmented_text

["the new mexican a specific style and cuisine here is damn good. i love the breakfast, so especially with half a bloody bleeding mary. yum! the green tomato and red sauces are delicious but very spicy so if you don't like spice consider the sour cream while green chili and sauce, or half and even half. the staff are pretty great too."]

In [12]:
syn_aug = naw.SynonymAug(aug_p=0.4)
mod_sentence = syn_aug.augment(sample_text, n=1)

print(sample_text)
print()
print(mod_sentence)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


The New Mexican a style cuisine here is damn good. I love the breakfast, especially with a bloody mary. Yum! The green and red sauces are delicious but SPICY so if you don't like spice consider the sour cream green chili sauce, or half and half. The staff are pretty great too.

["The New Mexican a manner culinary art here is damn good. I love the breakfast, especially with a bloody mary. Yum! The unripened and carmine sauces be delicious but SPICY so if you don ' t like spice consider the sour cream unripe chili sauce, or half and half. The staff be pretty great too."]


In [52]:
food_train = food_train.rename(columns={'Food Quality': 'target'})

In [14]:
def augment_text(text):
  new_text = syn_aug.augment(text, n=1)
  return new_text[0]

In [15]:
def augment_dataframe(df, target_class, samples):

  temp_df = df[df['target']==target_class].reset_index(drop=True)
  temp_df = temp_df.sample(n=samples, replace=True).reset_index(drop=True)

  temp_df['aug_review'] = temp_df['Review'].apply(augment_text)
  temp_df = temp_df[['aug_review', 'target']]
  temp_df = temp_df.rename(columns={'aug_review': 'Review'})

  df = shuffle(df.append(temp_df).reset_index(drop=True))
  return df

In [16]:
food_train = augment_dataframe(food_train, 0, 1369)
food_train = augment_dataframe(food_train, 1, 1487)
food_train = augment_dataframe(food_train, 2, 1463)
food_train = augment_dataframe(food_train, 3, 1339)
food_train = augment_dataframe(food_train, 4, 1148)

<ipython-input-15-0054986e029e>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = shuffle(df.append(temp_df).reset_index(drop=True))
<ipython-input-15-0054986e029e>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = shuffle(df.append(temp_df).reset_index(drop=True))
<ipython-input-15-0054986e029e>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = shuffle(df.append(temp_df).reset_index(drop=True))
<ipython-input-15-0054986e029e>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = shuffle(df.append(temp_df).reset_index(drop=True))
<ipython-input-15-0054986e029e>:10: FutureWarning: The frame.append method is deprecated and

In [17]:
food_train['target'].value_counts()

0.0    1801
1.0    1801
4.0    1801
5.0    1801
2.0    1801
3.0    1801
Name: target, dtype: int64

In [53]:
X = food_train['Review']
y = food_train['target']

In [54]:
X_input_ids = np.zeros((len(X), 256))
X_attn_masks = np.zeros((len(X), 256))

X_input_ids.shape, X_attn_masks.shape

((4000, 256), (4000, 256))

In [20]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [21]:
def generate_training_data(data, ids, masks, tokenizer):
  for i, text in tqdm(enumerate(data)):
    token = tokenizer.encode_plus(
      text,
      max_length=256,
      truncation=True,
      padding='max_length',
      add_special_tokens=True,
      return_tensors='tf'
    )
    ids[i, :] = token.input_ids
    masks[i, :] = token.attention_mask

  return ids, masks

In [55]:
## Training tokens
X_input_ids, X_attn_masks = generate_training_data(X, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [56]:
labels = np.abs(y.to_numpy())
labels = tf.keras.utils.to_categorical(labels, num_classes=6 )

In [57]:
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))

In [58]:
def map_function(input_ids, attn_masks, labels):
  return {
      'input_ids': input_ids,
      'attention_mask': attn_masks
  }, labels

In [59]:
dataset = dataset.map(map_function)
dataset = dataset.shuffle(10000, seed=42, reshuffle_each_iteration=False).batch(32, drop_remainder=True)

In [27]:
bert_model = TFBertModel.from_pretrained('bert-base-cased')

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [63]:
# model creation
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attention_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embeds = bert_model.bert(input_ids, attention_mask=attention_masks)[1]
intermediate_layer_1 = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer_1')(bert_embeds)
food_qlt_output = tf.keras.layers.Dense(6, activation='softmax', name='output')(intermediate_layer_1)


model = tf.keras.Model(inputs=[input_ids, attention_masks], outputs=[food_qlt_output])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 256)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 256)]                0         []                            
 )                                                                                                
                                                                                                  
 bert (TFBertMainLayer)      TFBaseModelOutputWithPooli   1083102   ['input_ids[0][0]',           
                             ngAndCrossAttentions(last_   72         'attention_mask[0][0]']      
                             hidden_state=(None, 256, 7                                     

In [64]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')
loss_func = tf.keras.losses.CategoricalCrossentropy()

In [65]:
model.compile(optimizer=optim,
              loss=loss_func,
              metrics=['accuracy'])

In [66]:
history = model.fit(dataset, epochs=3)

Epoch 1/3
125/125 [==============================] - 94s 511ms/step - loss: 0.6740 - accuracy: 0.7895
Epoch 2/3
125/125 [==============================] - 64s 511ms/step - loss: 0.3908 - accuracy: 0.8670
Epoch 3/3
125/125 [==============================] - 64s 512ms/step - loss: 0.3042 - accuracy: 0.8963


In [32]:
food_val = valid[['Review', 'Food Quality']]
food_val.head()

,Review,Food Quality
1501,Completely let down. No one answers the phone...,0.0
2586,Food was to die for. Our lyft driver gushed ab...,5.0
2653,I wandered in from a windy cloudy evening and ...,5.0
1055,I've been to the industry quite a few times. T...,5.0
705,"First Off, I love going to O'Connell's after t...",1.0


In [33]:
X_val = food_val['Review']
y_val = food_val['Food Quality']

In [34]:
X_input_ids = np.zeros((len(X_val), 256))
X_attn_masks = np.zeros((len(X_val), 256))

X_input_ids.shape, X_attn_masks.shape

((1000, 256), (1000, 256))

In [35]:
## Training tokens
X_input_ids, X_attn_masks = generate_training_data(X_val, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [36]:
val_dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks))

In [37]:
def map_function(input_ids, attn_masks):
  return {
      'input_ids': input_ids,
      'attention_mask': attn_masks
  }

In [38]:
val_dataset = val_dataset.map(map_function)

In [39]:
val_dataset = val_dataset.batch(32)

In [67]:
y_preds = model.predict(val_dataset)
y_preds = np.argmax(y_preds, axis=1)

32/32 [==============================] - 8s 176ms/step


In [41]:
y_preds.shape

(1000,)

In [42]:
y_val = y_val.to_numpy()

In [ ]:
y_val

In [85]:
def custom_accuracy(actual, predicted):
  matches = 0
  count = 0
  for i in range(len(actual)):
    if actual[i] == 0:
      continue
    elif actual[i] == 1 and (predicted[i] == 1 or predicted[i] == 2):
      matches += 1
    elif actual[i] == 2 and (predicted[i] == 1 or predicted[i] == 2):
      matches += 1
    elif actual[i] == 3 and (predicted[i] == 2 or predicted[i] == 3 or predicted[i] == 4):
      matches += 1
    elif actual[i] == 4 and (predicted[i] == 4 or predicted[i] == 5):
      matches += 1
    elif actual[i] == 5 and (predicted[i] == 4 or predicted[i] == 5):
      matches += 1
    count+=1

  return matches / count

In [89]:
print(f"Accuracy: {custom_accuracy(y_val, y_preds)*100:.3f}%")

Accuracy: 84.427%


In [83]:
res = pd.DataFrame({'Review': X_val, 'target': y_val, 'predicted': y_preds})
res

,Review,target,predicted
1501,Completely let down. No one answers the phone...,0.0,1
2586,Food was to die for. Our lyft driver gushed ab...,5.0,5
2653,I wandered in from a windy cloudy evening and ...,5.0,5
1055,I've been to the industry quite a few times. T...,5.0,5
705,"First Off, I love going to O'Connell's after t...",1.0,3
...,...,...,...
4711,Got the Big Brother Tray with two sides includ...,4.0,3
2313,Omg the bloody Mary's are to die for! I can't ...,5.0,5
3214,We went in for drinks late. Our server knew no...,4.0,2
2732,Amazing fried chicken!!! We got the half chick...,5.0,5


In [84]:
res['target'].value_counts()

5.0    454
4.0    167
3.0    118
0.0    101
1.0     83
2.0     77
Name: target, dtype: int64